In [1]:
import pandas as pd
import re

def remove_numbers_dots(value):
    cleaned = str(value).replace(".", "").replace(" ", "")
    return pd.to_numeric(cleaned, errors="coerce")

MONTHS = {
    "enero": 1, "febrero": 2, "marzo": 3, "abril": 4, "mayo": 5, "junio": 6,
    "julio": 7, "agosto": 8, "septiembre": 9, "setiembre": 9, "octubre": 10,
    "noviembre": 11, "diciembre": 12
}

def parse_periodo(s: str):
    s = str(s).strip().lower()
    m = re.match(r"(\d{1,2})\s+de\s+([a-zA-Zñ]+)\s+de\s+(\d{4})", s)
    if not m:
        return pd.NaT
    day = int(m.group(1))
    mon = MONTHS.get(m.group(2))
    year = int(m.group(3))
    return pd.Timestamp(year, mon, day) if mon else pd.NaT

def remove_aggregate_countries(df): 
    EXCLUDE_EXACT = {"Total", "Extranjera", "Apátrida", "No consta"}
    EXCLUDE_PREFIXES = ("De ", "País de ", "Países de ", "Resto de ", "Otros", "Sin ")

    mask = ~df["Nacionalidad"].str.strip().isin(EXCLUDE_EXACT)
    mask &= ~df["Nacionalidad"].str.strip().str.startswith(EXCLUDE_PREFIXES)
    base_countries = df[mask].copy()
    return base_countries
    

### **TOP 5 NACIONALIDADES**
### **EN ESPAÑA 2025**

In [29]:
# 1) Leemos el dataset de evolución de la migración en España
df = pd.read_csv('./evolucion_migracion.csv', sep=";")

# 2) Convertir "Total" (viene con separador de miles con puntos, ej: 49.128.297)
df["Total"] = df["Total"].apply(remove_numbers_dots)

# 3) Parsear "Periodo" (formato tipo: "1 de enero de 2025")
df["Periodo_dt"] = df["Periodo"].apply(parse_periodo)

# 4) Filtrar último periodo + "Todas las edades" + "Total"
latest = df["Periodo_dt"].max()
base_countries = df[
    (df["Periodo_dt"] == latest) &
    (df["Grupo quinquenal de edad"].str.strip() == "Todas las edades") &
    (df["Sexo"].str.strip() == "Total")
].copy()

# 5) (Opcional pero recomendado) quitar agregados para quedarte con países
total_population = (
        df.loc[df["Nacionalidad"].str.strip() == "Total", "Total"]
        .iloc[0]
    )

#base_countries = remove_aggregate_countries(base_countries)
base_countries = base_countries[~base_countries["Nacionalidad"].isin(["Total", "Extranjera"]) ]
base_countries.loc[base_countries["Nacionalidad"] == "De Sudamérica", "Nacionalidad"] = "De América"

# 6) Calcular porcentaje
base_countries["pct_total"] = (base_countries["Total"] / total_population) * 100

# 7) Top 5 porcentajes
top5_pct = (
    base_countries
    .sort_values("pct_total", ascending=False)
    .head(5)[["Nacionalidad", "Total", "pct_total"]]
    .reset_index(drop=True)
)

top5_pct["pct_total"] = top5_pct["pct_total"].round(3)

print(f"Último periodo: {latest.date()}")
print(top5_pct)

Último periodo: 2025-01-01
                      Nacionalidad       Total  pct_total
0                         Española  42216326.0     85.931
1                       De América   1958005.0      3.985
2  País de la UE27_2020 sin España   1757955.0      3.578
3                        De Africa   1367584.0      2.784
4                        Marruecos    968999.0      1.972


## **TOP 5 NACIONALIDADES**
## **MÁS PELIGROSAS** 
## **EN ESPAÑA** 

In [36]:
# 1) Leemos el dataset de evolución de la migración en España

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

df = pd.read_csv('./delicuencia_nacionalidad.csv', sep=";")

# 2) Normalizar la columna Total para poder sumar por nacionalidad
df["Total"] = df["Total"].apply(remove_numbers_dots)

df = df[df["Nacionalidad"] != "Total"]

# 3) Suma de delitos por nacionalidad (todas las filas)
delitos_por_nacionalidad = (
    df.groupby("Nacionalidad", as_index=False)["Total"].sum()
    .sort_values("Total", ascending=False)
    .reset_index(drop=True)
)

# 4) Calcular porcentaje sobre el total general
total_general = delitos_por_nacionalidad["Total"].sum()
delitos_por_nacionalidad["pct_total"] = (delitos_por_nacionalidad["Total"] / total_general) * 100
delitos_por_nacionalidad["pct_total"] = delitos_por_nacionalidad["pct_total"].round(2)

df_resultado = pd.merge(
    delitos_por_nacionalidad,
    base_countries,
    on="Nacionalidad",
    how="inner"
).sort_values("Total_x", ascending=False)

df_resultado.head(3)

df.iloc[:, :3].apply(lambda s: sorted(s.dropna().unique()))


Tipo de Delito: Nivel 1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 